In [6]:
import json
import csv
import re
from datetime import date,datetime,timedelta
from pathlib import Path
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
!pip install yfinance 
import yfinance as yf

In [12]:
def stock_data(ticker):
    start_date = '2022-03-01'
    end_date = date.today()+timedelta(days=1)
    
    # 1. Request data:
    data = yf.download(ticker, 
                      start=start_date, 
                      end=end_date,
                      interval='1d', 
                      progress=False)
    
    # 2. Feature Engineering:
    # data['Percent Price Change Within Period'] = ((data['Close'] - data['Open'])/data['Open'])*100
    # data['Change in Close Price'] = data['Close'] - data['Close'].shift(1)
    # data['Scaled Delta Close'] = data['Change in Close Price']/(data['Close'].mean())
    # data['Scaled Volume'] = data['Volume']/data['Volume'].mean()
    # data_SMA = data['Adj Close'].rolling(window=3).mean().shift(1)
    # data['SMA(3)'] = data_SMA
    
    # Time Series Variables Creation 
    # t+1, t-1, t-2, t-3 Closes
    # t+1, t-1, t-2, t-3 Volume
    data['Volume-1d'] = data['Volume'].shift(1)
    data['Volume-2d'] = data['Volume'].shift(2)
    data['Volume-3d'] = data['Volume'].shift(3)
    
    data['Close-1d'] = data['Close'].shift(1)
    data['Close-2d'] = data['Close'].shift(2)
    data['Close-3d'] = data['Close'].shift(3)

    # TARGET CREATION
    data['Close-tomorrow'] = data['Close'].shift(-1)
    
    # Reset Index
    data.reset_index(inplace=True)
    
    #data['Datetime']=data['Datetime'].dt.tz_convert('America/Montreal').dt.tz_localize(None)
    
    data.drop(['Adj Close', 'Open', 'High', 'Low'],axis=1,inplace=True)
    
    #3. Export data:
    #f_name = ticker + "_data"
    # data.to_csv('~/LighthouseLabs-Final/Dataset/1. Stock_Data/' + f_name + ".csv")
    # print('Data saved!')
    
    return data

In [13]:

# Pass in ticker and return DataFrame
technical_analysis_df = pd.DataFrame(stock_data('TSLA'))
technical_analysis_df

,Date,Close,Volume,Volume-1d,Volume-2d,Volume-3d,Close-1d,Close-2d,Close-3d,Close-tomorrow
0,2022-03-01,864.369995,24922300,NaN,NaN,NaN,NaN,NaN,NaN,879.890015
1,2022-03-02,879.890015,24881100,24922300.0,NaN,NaN,864.369995,NaN,NaN,839.289978
2,2022-03-03,839.289978,20541200,24881100.0,24922300.0,NaN,879.890015,864.369995,NaN,838.289978
3,2022-03-04,838.289978,22333200,20541200.0,24881100.0,24922300.0,839.289978,879.890015,864.369995,804.580017
4,2022-03-07,804.580017,24164700,22333200.0,20541200.0,24881100.0,838.289978,839.289978,879.890015,824.400024
5,2022-03-08,824.400024,26799700,24164700.0,22333200.0,20541200.0,804.580017,838.289978,839.289978,858.969971
6,2022-03-09,858.969971,19728000,26799700.0,24164700.0,22333200.0,824.400024,804.580017,838.289978,838.299988
7,2022-03-10,838.299988,19549500,19728000.0,26799700.0,24164700.0,858.969971,824.400024,804.580017,795.349976
8,2022-03-11,795.349976,22272800,19549500.0,19728000.0,26799700.0,838.299988,858.969971,824.400024,766.369995
9,2022-03-14,766.369995,23717400,22272800.0,19549500.0,19728000.0,795.349976,838.299988,858.969971,801.890015


In [14]:
# Set date as index
technical_analysis_df.set_index('Date', inplace = True)

In [17]:
# Show Index
technical_analysis_df.head()

,Close,Volume,Volume-1d,Volume-2d,Volume-3d,Close-1d,Close-2d,Close-3d,Close-tomorrow
Date,,,,,,,,,
2022-03-01,864.369995,24922300,NaN,NaN,NaN,NaN,NaN,NaN,879.890015
2022-03-02,879.890015,24881100,24922300.0,NaN,NaN,864.369995,NaN,NaN,839.289978
2022-03-03,839.289978,20541200,24881100.0,24922300.0,NaN,879.890015,864.369995,NaN,838.289978
2022-03-04,838.289978,22333200,20541200.0,24881100.0,24922300.0,839.289978,879.890015,864.369995,804.580017
2022-03-07,804.580017,24164700,22333200.0,20541200.0,24881100.0,838.289978,839.289978,879.890015,824.400024


In [19]:
# Write CSV to File
csvpath = Path("./technical_analysis_df.csv")
technical_analysis_df.to_csv(csvpath)